In [1]:
import sys
sys.path.insert(1, '/scratch/cinthiasouza/mv-text-summarizer')

import itertools
import re

import glob, os
import pandas as pd
import json
import spacy
import nltk
import numpy as np
import json
#import seaborn as sns
import pickle

import math


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler


import warnings
warnings.filterwarnings("ignore")

path_base = "/scratch/cinthiasouza/mv-text-summarizer"
path_to_read="/scratch/cinthiasouza/mv-text-summarizer/result/{}/{}_*.csv"

In [16]:
import spacy

ModuleNotFoundError: No module named 'spacy'

In [31]:
%load_ext autoreload
%autoreload 2

#from src import preprocess
#from src import extract_features
#from src import tokenizer
#from src import create_features_df
from src import transform_data
#from src import loader
#from src import utils
#from src import ensemble_tree_models
#from src import tunning_hyperparametrs as th
#from src import mlp_regressor
#from src import mlp_classifier
from src import summarization
from src import normalization
from src import ensemble_tree_models as classifiers
from src import utils_classification as utils_clf
from src import evaluate_classifiers as ev
from src import prepare_data
from src import display_results as dr
import joblib
from joblib import Parallel, delayed
#from tensorflow.keras.utils import to_categorical
from src import pipeline_extract_features as pef

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from tensorflow import keras
from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision

In [4]:
def load_keras_model(path_to_save, name_model, section):

    json_file = open('{}/{}_{}.json'.format(path_to_save, name_model, section), 'r')
    model = json_file.read()
    json_file.close()
    model = model_from_json(model)
    model.load_weights('{}/{}_{}.h5'.format(path_to_save, name_model, section))
    print("Loaded model from disk")
    model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(
                            learning_rate=0.001), metrics=[keras.metrics.Precision()])
    
    return model

In [5]:
def load_classifiers(sections, path_to_read, name_models):

    models = {}
    
    for section in sections:
        
        aux = {}
        
        for name_model in name_models:
            if name_model != 'mlp':
                aux[name_model] = joblib.load('{}/{}_{}.pkl'.format(path_to_read, name_model,  section))
            elif (name_model == 'mlp') or (name_model == 'embed_mlp') or (name_model == 'mv_mlp_bert') :
                aux[name_model] = load_keras_model(section, path_to_read)
            
        models[section] = aux
        
    return models

In [6]:
def remove_ascii(text):
    
    try:
        return re.sub(r'[^\x00-\x7F]+', '', text)
    except:
        return " "

In [7]:
def pipeline_summarization(
    features, scores, references, predictions, section, name_models,
    summ_items, k=3, sort_scores=True, proba=False, ascending=False):
    
    X_test = features
    y_test = scores
    
    vfunc = np.vectorize(remove_ascii)
    X_test['sentences'] = vfunc(X_test['sentences'])

    df_proba = summarization.create_df(name_models, X_test, y_test['rouge_1'], predictions, section, proba=proba)
    
    if proba:
        df = summarization.binarize_proba(df_proba.copy(), name_models, k, sort_scores, ascending)
    else:
        df = df_proba.copy()
        
    summaries = summarization.create_summaries(df, references, summ_items, name_models)
    result = summarization.evaluate_summariesv2(summaries, name_models, metrics=['rouge-1', 'rouge-2', 'rouge-l'])
    
    return df_proba, df, summaries, result

In [8]:
def create_df_v2(name_models, x_summ):
    
    df = pd.DataFrame({'sentences': x_summ['sentences'],
                       'rouge_1': list(x_summ['rouge_1']),
                       'articles': x_summ['articles']})
    
    for name_model in name_models:
            df[name_model] = list(x_summ['rouge_1'])
            
    return df

In [9]:
def summarization_target(
    df, references, name_models, summ_items, sort_scores=False, proba=False, ascending=False):
    
    df = create_df_v2(name_models, df.copy())

    df = summarization.binarize_proba(df.copy(), name_models, 3, sort_scores, ascending)

    summaries = summarization.create_summaries(df, references, summ_items, name_models)
    result = summarization.evaluate_summariesv2(summaries,  name_models, metrics=['rouge-1', 'rouge-2', 'rouge-l'])
    
    return df, summaries, result

In [28]:
def load_predict_models(dataset, sections, name_models, columns, path_to_save, embed_path, num_test, index_Xtest):
     
    predictions_proba = {}
    models = {}

    for section in sections:

        aux = {}
        aux_models = {}
        
        X_test_embed = pd.read_csv("dataset/{}_{}_test.csv".format(embed_path, section))
        X_test_embed = X_test_embed[columns]
        
        X_test_features = dataset[section][index_Xtest]
        
        for name_model in name_models:

            #Load
            if (name_model != 'mlp') and (name_model != 'mlp_embed') and (name_model != 'mv_mlp_bert'):
                model = joblib.load('{}/test_{}/{}_{}.pkl'.format(path_to_save, num_test, name_model, section))
            else :
                model = load_keras_model(path_to_save, name_model, section)

            #predict
            if (name_model == 'mv_mlp_bert') or (name_model == 'mv_mlp'):
                y_pred = model.predict([X_test_embed, X_test_features])
            elif name_model == "mlp":
                y_pred = model.predict(X_test_features)
            elif name_model == 'mlp_embed':
                y_pred = model.predict(X_test_embed)
            else:
                y_pred = model.predict_proba(X_test_features)


            aux[name_model] = y_pred
            aux_models[name_model] = model

        predictions_proba[section]= aux
        models[section] = aux_models
        
    return predictions_proba, models

In [11]:
def format_data_to_summarize(dataset, section):
    
    df = dataset[section][5].reset_index(drop=True)
    features = df[['sentences', 'articles']]
    scores = pd.DataFrame()
    scores['rouge_1'] = df['rouge_1']
    
    return features, scores

In [12]:
with open('autoencoder_64dim/dataset_{}.pkl'.format('features'), 'rb') as fp:
    dataset = pickle.load(fp)
    
columns_name = ['text_rank', 'lex_rank', 'count_one_gram', 'count_two_gram', 'count_three_gram',
        'count_article_keywords',
       'tf-isf', 'position_score', 'paragraph_score',
       'length_score', 'pos_score', 'ner_score', 'dist_centroid']

In [13]:
sections=['introduction', 'materials', 'conclusion']

In [14]:
name_models = ['knn', 'rf', 'ab', 'gb', 'cb']

In [15]:
columns = list(range(0, 383))
columns = list(map(str, columns))

embed_path = 'embed_bert'

In [16]:
summ_items = pd.read_csv("dataset/indices_summ.csv")['summ']
references_df = pd.read_csv("dataset/references_df.csv")
features_intro = pd.read_csv('dataset/dataset_introduction.csv')
features_mat = pd.read_csv('dataset/dataset_materials.csv')
features_conc = pd.read_csv('dataset/dataset_conclusion.csv')

In [44]:
tests = {'mv_models_64dim':'mv'}

In [45]:
def combine_summaries_eval(
    summaries_intro, summaries_mat, summaries_conc, references_df,
    name_models = ['knn', 'rf', 'ab', 'gb', 'cb'], metrics=['rouge-1', 'rouge-2', 'rouge-l']):
    
    summaries = summarization.combine_three_summ(
        summaries_intro, summaries_mat, summaries_conc, references_df, name_models)
    result = summarization.evaluate_summariesv2(summaries, name_models, metrics)
    
    return summaries, result

In [46]:
def main (
    dataset, tests, num_test, references_df, sections, name_models, columns, embed_path):
    
    for name_test, approach in tests.items():

        if approach == 'sv':
            index_Xtest=1
            index_ytest=3
        elif approach == 'mv':
            index_Xtest=-1
            index_ytest=3

        print(num_test)

        path_to_save = "/scratch/cinthiasouza/mv-text-summarizer/notebook/{}".format(name_test)

        predictions_proba, models = load_predict_models(
            dataset, sections, name_models, columns, path_to_save, embed_path, num_test, index_Xtest=index_Xtest)

        predictions, predictions_proba, results = ev.create_reports(
            models, dataset, name_models, index_Xtest=index_Xtest, index_ytest=index_ytest, verbose=False)

        if not os.path.exists("{}/test_{}/summaries".format(path_to_save, num_test)):
                os.makedirs("{}/test_{}/predictions".format(path_to_save, num_test))
                os.makedirs("{}/test_{}/summaries".format(path_to_save, num_test))

        aux = {}
        for section in sections:

            features, scores = format_data_to_summarize(dataset, section)

            proba_ex1, df_ex1, summaries_ex1, result_ex1 = pipeline_summarization(
                features, scores, references_df, predictions_proba, section, name_models,
                summ_items, sort_scores=True, proba=True, ascending=False)

            aux[section] = summaries_ex1

            result_ex1.to_csv("{}/test_{}/summaries/{}.csv".format(path_to_save, num_test, section), index=False)
            proba_ex1.to_csv("{}/test_{}/predictions/{}.csv".format(path_to_save,num_test, section), index=False)
            df_ex1.to_csv("{}/test_{}/predictions/df_{}.csv".format(path_to_save, num_test, section), index=False)

        summaries, result_comb = combine_summaries_eval(
            aux['introduction'], aux['materials'], aux['conclusion'], references_df)

        result_comb.to_csv("{}/test_{}/summaries/{}.csv".format(path_to_save, num_test, 'comb'), index=False)

In [47]:

from joblib import Parallel, delayed

Parallel(n_jobs=5)(delayed(main)(dataset, tests, num_test, references_df, sections, name_models, columns, embed_path) for num_test in range(1, 31))
 

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [60]:
for num_test in range(1, 31):
    for name_test, approach in tests.items():

        if approach == 'sv':
            index_Xtest=1
            index_ytest=3
        elif approach == 'mv':
            index_Xtest=-1
            index_ytest=3

        print(num_test)

        path_to_save = "/scratch/cinthiasouza/mv-text-summarizer/notebook/{}".format(name_test)

        predictions_proba, models = load_predict_models(
            dataset, sections, name_models, columns, path_to_save, embed_path, index_Xtest=index_Xtest)

        predictions, predictions_proba, results = ev.create_reports(
            models, dataset, name_models, index_Xtest=index_Xtest, index_ytest=index_ytest, verbose=False)

        if not os.path.exists("{}/test_{}/summaries".format(path_to_save, num_test)):
                os.makedirs("{}/test_{}/predictions".format(path_to_save, num_test))
                os.makedirs("{}/test_{}/summaries".format(path_to_save, num_test))

        aux = {}
        for section in sections:

            features, scores = format_data_to_summarize(dataset, section)

            proba_ex1, df_ex1, summaries_ex1, result_ex1 = pipeline_summarization(
                features, scores, references_df, predictions_proba, section, name_models,
                summ_items, sort_scores=True, proba=True, ascending=False)

            aux[section] = summaries_ex1

            result_ex1.to_csv("{}/summaries/{}.csv".format(path_to_save, section), index=False)
            proba_ex1.to_csv("{}/predictions/{}.csv".format(path_to_save, section), index=False)
            df_ex1.to_csv("{}/predictions/df_{}.csv".format(path_to_save, section), index=False)

        summaries, result_comb = combine_summaries_eval(
            aux['introduction'], aux['materials'], aux['conclusion'], references_df)

        result_comb.to_csv("{}/summaries/{}.csv".format(path_to_save, 'comb'), index=False)

mv_models_64dim
